# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)


From this point we will use __%%sql__ magic

In [4]:
%load_ext sql
%sql sqlite:///wine.db

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [5]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [6]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [7]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [8]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
dummy:
  0: DEGRE(NUM)


In [9]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [10]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [11]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [12]:
%%sql drop view if exists Wine;
    CREATE VIEW Wine AS
    SELECT NV, CRU, DEGRE, MILL
    FROM Master1
    where nv is not null;
    
    drop view if exists Producer;
    create view Producer as
    select NP, NOM, PRENOM, REGION
    from Master1
    where np is not null;
    
    drop view if exists Product;
    create view Product as
    select NP, NV, QTE
    from Master1
    where np is not null or nv is not null;


 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

#### Exercise 1.4

Create the new tables from Master2:

In [13]:
%%sql drop view if exists Buyer;
    CREATE VIEW Buyer AS
    SELECT NB, NOM, PRENOM, TYPE
    FROM Master2
    where nb is not null;
    
    drop view if exists BuyedWine;
    create view BuyedWine as
    select NV, CRU, DEGRE, MILL
    from Master2
    where nv is not null;
    
    drop view if exists Sale;
    create view Sale as
    select NB, NV, DATES, LIEU, QTE
    from Master2
    where nb is not null or nv is not null or dates is not null or lieu is not null;
    
    drop view if exists Region;
    create view Region as
    select lieu, region
    from Master2
    where lieu is not null;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [14]:
%%sql
select distinct TYPE
from buyer


 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [15]:
%%sql
select distinct region
from Producer pd, Wine w, Product pt
where pd.NP = pt.NP
    and w.NV = pt.NV
    and w.cru = "Pommard" or w.cru = "Brouilly"

 * sqlite:///wine.db
Done.


REGION
Alsace
Bourgogne
Beaujolais
Jura
Rhone
Provence
Corse
Languedoc
Pyrenees
Sud


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [16]:
%%sql
select distinct region
from Producer pd, Wine w, Product pt
where pd.NP = pt.NP
    and w.NV = pt.NV
    and w.cru = "Pommard"

intersect

select distinct region
from Producer pd, Wine w, Product pt
where pd.NP = pt.NP
    and w.NV = pt.NV
    and w.cru = "Brouilly"

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [17]:
%%sql
select cru, mill, sum(qte) as Num
from Wine w, Product p
where w.NV = p.Nv
group by w.cru

 * sqlite:///wine.db
Done.


CRU,MILL,Num
Arbois,1980,6594
Auxey Duresses,1914,2500
Beaujolais Primeur,1983,80
Beaujolais Villages,1979,9790
Bellet,1976,45
Blanquette de Limoux,1978,110
Bourgogne Passe Tout Grains,1979,350
Brouilly,1975,500
Cassis,1978,58
Chambolle Musigny,1968,120


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [18]:
%%sql
select Wine.NV, count(Producer.NP) as num
from Wine, Producer, Product
where Wine.nv = Product.nv
    and Producer.np = Product.np
group by Wine.nv
having count(Producer.np) > 3

 * sqlite:///wine.db
Done.


NV,num
1,8
2,5
8,4
11,5
12,21
14,4
16,6
17,6
18,4
20,5


#### Exercise 2.6

Which producers have not produced any wine?

In [19]:
%%sql
select Producer.np
from Producer, Product
where Producer.np = Product.np
group by Product.np
having count(Product.nv) = 0

 * sqlite:///wine.db
Done.


NP
3
6
8
16
17
18
19
20
21
26


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [20]:
%%sql
select distinct Buyer.nb, Buyer.nom, Buyer.prenom
from Buyer, BuyedWine, Sale
where Buyer.nb = Sale.nb
    and BuyedWine.nv = Sale.nv
    and BuyedWine.mill = '1980'

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
45,Giono,Jean
44,Gide,Andre
50,Lautreamont,None
8,Aragon,Louis
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [21]:
%%sql
select distinct Buyer.nb, Buyer.nom, Buyer.prenom
from Buyer, BuyedWine, Sale
except
select distinct Buyer.nb, Buyer.nom, Buyer.prenom
from Buyer, BuyedWine, Sale
where Buyer.nb = Sale.nb
    and BuyedWine.nv = Sale.nv
    and BuyedWine.mill = '1980'

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [22]:
%%sql
select distinct Buyer.nb, Buyer.nom, Buyer.prenom
from Buyer, BuyedWine, Sale
where Buyer.nb = Sale.nb
    and BuyedWine.nv = Sale.nv
    and BuyedWine.mill = '1980'
except
select distinct Buyer.nb, Buyer.nom, Buyer.prenom
from Buyer, BuyedWine, Sale
where Buyer.nb = Sale.nb
    and BuyedWine.nv = Sale.nv
    and BuyedWine.mill != '1980'

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [23]:
%%sql
select distinct nv, cru, mill, degre
from Wine
where mill = '1980'

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [24]:
%%sql
select distinct BuyedWine.nv, BuyedWine.cru, BuyedWine.mill,BuyedWine.degre
from BuyedWine,Sale
where Sale.nb =2 AND BuyedWine.mill ='1980' and Sale.nv = BuyedWine.nv

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [28]:
%%sql
select Buyer.nb, Buyer.nom, Buyer.prenom
from Buyer
where nb in(
    select temp.nb
    from Sale temp
    where temp.nv in(
        select temp.nv
        from Wine
        where mill = '1980'
    )
    group by temp.nb
    having count(temp.nb) = (
        select count(BuyedWine.nv)
        from BuyedWine
        where mill = '1980' 
    )  
)

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
